**Комментарий из будущего** <br>
Тут живет подобие нейросети на BERT. В теории первоначальная цель была достигнута, модель работала на удаленном сервере на обозначенных ресурсах. Но попросили сократить :) Был убран BERT, который много жрет, и сама нейросеть тоже пошла под ножик из-за размеров торча. <br>
По итогу в этом файле только формирование ембедингов от Берта, даже нейросетка переехала в конец 5 части :))

In [14]:
import pandas as pd
import numpy as np

import spacy
import pickle

import transformers as ppb

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm, notebook

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [20]:
RAND = sum(ord(x) for x in 'NEVER SURRENDER')
tqdm.pandas()

HIDDEN_N = 530  # 800 * 2/3
ETA = 0.001
BATCH = 200
EPOCH = 1_000

In [114]:
df = pd.read_csv('tocolab.csv', index_col=[0])
df.columns = df.columns.str.replace('(.)([A-Z])', r'\1_\2', regex=True).str.lower()

In [115]:
df = df[:-100]

In [116]:
df_train, df_valid = train_test_split(df, test_size=0.1, random_state=RAND)

In [117]:
print(df_train.shape, df_valid.shape)

(220201, 4) (24467, 4)


(чуть позже переделана предобработка, словарь формируется автоматически, но результат аналогичен)

In [40]:
#load dict
with open('nn/location_dict.pkl', 'rb') as f:
    location_dict = pickle.load(f)

In [41]:
class LocationCategory(BaseEstimator, TransformerMixin):
    
    def __init__(self, location_dict):
        #load dict
        self.location_dict = location_dict
        

    def fit(self, X, y=None):
        #no need to fit, only transform
        return self

    def transform(self, X):
        return pd.DataFrame(X.apply(lambda x: location_dict.get(x,'few')))

In [42]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
])

location_transform = ColumnTransformer([('loc', LocationCategory(location_dict), 'location_normalized')], 
                                       remainder='passthrough')

final_pipe = Pipeline([
    ('loc', location_transform),
    ('prep', cat_pipe)
   
])

In [17]:
df_pipe = df_train[['location_normalized', 'category']]

In [19]:
tt = final_pipe.fit(df_pipe)

In [20]:
train_cat_features = tt.transform(df_train)
valid_cat_features = tt.transform(df_valid)

Категории готовы, Берт.

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
#ФРААААНКЕНШТЕЙН :) поэтому не особо упорядочено, делалось кусками. 
def bert_transform(data: pd.Series, 
                   device: torch.device,
                   batch_size: int = 0) -> pd.DataFrame:
    
    if not batch_size:
        batch_size = data.shape[0]
    
    #distilbert load. 
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, 
                                                        ppb.DistilBertTokenizer, 
                                                        'distilbert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model_bert = model_class.from_pretrained(pretrained_weights).to(device)
    
    b_token = data.progress_apply((lambda x: tokenizer.encode(x, 
                                                     add_special_tokens=True,
                                                     truncation=True)))
    max_len = max(b_token.map(len))
    padded = np.array([i + [0]*(max_len-len(i)) for i in b_token.values])
    attention_mask = np.where(padded != 0, 1, 0)
    embeddings = []
    start_ = 0
    stop_ = batch_size
    for j in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[start_:stop_]).to(device)  
        attention_mask_batch = torch.LongTensor(attention_mask[start_:stop_]).to(device)  
        with torch.no_grad():
            batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        start_ += batch_size
        stop_ += batch_size
    bert_features = np.concatenate(embeddings)
    return pd.DataFrame(bert_features)

In [26]:
try:
    bert_features = pd.read_csv('full-bert-train.csv', index_col=[0])
except:
    bert_features = bert_transform(df_train['clear_text'], device, BATCH)
    pd.DataFrame(bert_features).to_csv('full-bert-train.csv')

In [25]:
try:
    bert_features_valid  = pd.read_csv('full-bert-valid.csv', index_col=[0])
except:
    bert_features_valid  = bert_transform(df_valid['clear_text'], device, BATCH)
    pd.DataFrame(bert_features_valid).to_csv('full-bert-valid.csv')

In [28]:
X_train = pd.concat([bert_features, pd.DataFrame(train_cat_features)], axis=1)
X_train = X_train.dropna()
X_valid = pd.concat([bert_features_valid, pd.DataFrame(valid_cat_features)], axis=1)
X_valid = X_valid.dropna()

_________________
Целевой. Тут получился без логарифма


In [118]:
#df_train['salary_normalized'] = np.log1p(df_train['salary_normalized'])
try:
    #load 
    with open("full_minmax.pkl", "rb") as f:
        minmax = pickle.load(f)
except:
    minmax = MinMaxScaler().fit(df_train['salary_normalized'].values.reshape(-1, 1))
    #save 
    with open("full_minmax.pkl", "wb") as f:
        pickle.dump(minmax, f)

In [119]:
y_train = minmax.transform(df_train['salary_normalized'].values.reshape(-1, 1))
y_valid = minmax.transform(df_valid['salary_normalized'].values.reshape(-1, 1))

In [120]:
y_train = y_train[:X_train.shape[0]]
y_valid = y_valid[:X_valid.shape[0]]

_________________

In [32]:
n_input = X_train.shape[1]
torch.manual_seed(RAND)

In [98]:
device

device(type='cuda')

In [24]:
# плохо у меня с фантазией на названия
# _____________________
# Ist das der Zerstörer oder der Schöpfer?

class TextNet(nn.Module):
    def __init__(self, n_input, n_hidden, layers):
        super(TextNet, self).__init__()
        
        self.input_layers = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Dropout(0.1),
           # nn.ReLU()
        )
        
        
        mid_layers = []
        for _ in range(layers):
            mid_layers.append(nn.Linear(n_hidden, n_hidden))
            mid_layers.append(nn.Dropout(0.1))
            mid_layers.append(nn.ReLU()) 
        self.mid_layers = nn.Sequential(*mid_layers)
        
        
        self.out_layres = nn.Sequential(       
            nn.Linear(n_hidden, int(n_hidden/2)),
            nn.Sigmoid(), #nn.ReLU(), #
            nn.Linear(int(n_hidden/2), 1)
        )
    
          
    def forward(self, x):
        y = self.input_layers(x)
        y = self.mid_layers(y)
        y = self.out_layres(y)
        return y

In [149]:
nn_model = TextNet(n_input, HIDDEN_N, 1)
if torch.cuda.is_available():
    nn_model = nn_model.to(device)

In [130]:
nn_model

TextNet(
  (input_layers): Sequential(
    (0): Linear(in_features=800, out_features=530, bias=True)
    (1): Dropout(p=0.1, inplace=False)
  )
  (mid_layers): Sequential(
    (0): Linear(in_features=530, out_features=530, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): ReLU()
  )
  (out_layres): Sequential(
    (0): Linear(in_features=530, out_features=265, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=265, out_features=1, bias=True)
  )
)

In [150]:
def create_tensor_dataloader(features: pd.DataFrame, 
                             target: pd.Series,
                             batch_: int = BATCH,
                             split: float=0) -> DataLoader:
    
    #create dataset
    features = torch.tensor(features.values, device=device,  dtype=torch.float)
    target = torch.tensor(target, device=device,  dtype=torch.float) #labels
        
    dataset = TensorDataset(features, target)
    
    if split:
        train, valid = train_test_split(dataset, test_size=split, random_state=RAND)
        train = DataLoader(train, batch_size=batch_, drop_last=True) 
        valid = DataLoader(valid, batch_size=batch_, drop_last=True) 
        return train, valid
    else:
        return DataLoader(dataset, batch_size=batch_, drop_last=True) 

In [151]:
tensor_train = create_tensor_dataloader(X_train, y_train)
tensor_valid = create_tensor_dataloader(X_valid, y_valid)

In [152]:
### не финальная моделька. просто какая-то из.

In [153]:
nn_model.train()
loss_function =  nn.L1Loss() # nn.MSELoss() # 
optimizer = torch.optim.Adam(nn_model.parameters(), lr=ETA)
  
for epoch in tqdm(range(EPOCH)):   
    for inputs, targets in tensor_train:
        optimizer.zero_grad()
        outputs = nn_model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step() 

100%|██████████| 200/200 [13:04<00:00,  3.92s/it]


In [154]:
pred = np.array([])
real = np.array([])
nn_model.eval()
with torch.no_grad():
    for inputs, targets in tqdm(tensor_valid):  
        
        output = nn_model(inputs)
            
        pred = np.append(pred, output.detach().cpu())
        real = np.append(real, targets.detach().cpu())
       # print(f'R2 = {r2_score(real, pred)}')

print('__________________')
print(f'MSE = {mean_squared_error(real, pred)}')
print(f'R2 = {r2_score(real, pred)}')
print(f'MAE = {mean_absolute_error(real, pred)}')

100%|██████████| 122/122 [00:00<00:00, 537.86it/s]

__________________
MSE = 0.003160029237172051
R2 = 0.6002499147026861
MAE = 0.03578555290501672


MSE и MAE с учетом minmax, не очень показательно получилось :)

In [155]:
torch.save(nn_model.state_dict(), 'nn_model_state_5.pth')
torch.save(nn_model, 'nn_model_5.pth')